I have uploaded the train.jsonl, test.jsonl and dev.jsonl on my google drive.

In [0]:
!pip install json-lines

In [0]:
import json_lines
import json
import pandas as pd
import requests
import numpy as np
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model
import keras.layers
from sklearn.preprocessing import StandardScaler
from keras import optimizers
from sklearn.preprocessing import OneHotEncoder


In [0]:
#getting dataset from git repo. I uploaded the files in the Text-Entailment repository

req1 = requests.get('https://raw.githubusercontent.com/tanishkasingh9/Text-Entailment/master/dev.jsonl')
c1 = req1.content
req2 = requests.get('https://raw.githubusercontent.com/tanishkasingh9/Text-Entailment/master/train.jsonl')
c2 = req2.content
req3 = requests.get('https://raw.githubusercontent.com/tanishkasingh9/Text-Entailment/master/test.jsonl')
c3 = req3.content

#print(c)

dev = pd.read_json( c1, lines=True)
train = pd.read_json( c2, lines=True)
test = pd.read_json( c3, lines=True)
print(dev)

      

    annotator_labels       genre     gold_label   pairID  promptID  \
0       [entailment]      travel     entailment    8608e      8608   
1    [contradiction]       slate  contradiction   51889c     51889   
2          [neutral]   telephone        neutral   72610n     72610   
3    [contradiction]      travel  contradiction   58952c     58952   
4    [contradiction]     fiction  contradiction   61024c     61024   
5          [neutral]   telephone        neutral   60546n     60546   
6          [neutral]   telephone        neutral   27530n     27530   
7       [entailment]   telephone     entailment   30033e     30033   
8    [contradiction]     fiction  contradiction   35087c     35087   
9       [entailment]     fiction     entailment   62291e     62291   
10   [contradiction]     fiction  contradiction   23916c     23916   
11         [neutral]       slate        neutral    9967n      9967   
12         [neutral]       slate        neutral  115147n    115147   
13      [entailment]

In [0]:
#Downloading GLOVE
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2019-04-08 18:32:51--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-04-08 18:32:52--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  17.0MB/s    in 64s     

2019-04-08 18:33:56 (12.9 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
#Creating word graph 
word2vec = {}
with open('glove.6B.50d.txt', encoding="utf8") as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Found 400000 word vectors.


In [0]:
#removing '-' from test labels and replacing with 'neutral'
print(test.shape)
test = test[test.gold_label != '-']
print(test.shape)
    

(1000, 11)
(990, 11)


In [0]:
def word_embeddings(dataset):
  #create embedding matrix from premise and hypothesis
 
  
  premise_embedding_matrix =np.empty((750,), float)
  hypothesis_embedding_matrix =np.empty((750,), float)
  premise_sentence_length = []
  hypothesis_sentence_length = []
  embedding_vec_np=[]
  
  
  #tokenize sentences
  for row in dataset['sentence1']:
    flatten = []
    word_sent = row.split()
    tokenizer = Tokenizer( filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')
    tokenizer.fit_on_texts(word_sent)
    word_sent_dict = tokenizer.word_index
    premise_sentence_length = np.append(premise_sentence_length,len(word_sent_dict))
    #print(neg)
    #remove words that do not have word vector in glove
    t=0
    for word,i in word_sent_dict.items():
      #get the vectors for each words
      embedding_vector = word2vec.get(word)
   

      if embedding_vector is not None:
        # words not found in embedding index will be all zeros.
        flatten= np.asarray(flatten)
        #print(premise_1.shape)
        flatten = np.append(flatten,embedding_vector)
        t= t+1
        
        
    #print(flatten.shape) 
    zero_pad = 15 - t
    arr = [0]*50*zero_pad
    flatten = np.append(flatten, arr)
    #print(flatten.shape) 
    premise_embedding_matrix = np.vstack((premise_embedding_matrix,flatten))
  premise_embedding_matrix = np.delete(premise_embedding_matrix, 0, 0)
          
  #print(premise_embedding_matrix.shape)
  #print(len(premise_sentence_length))

  
  for row in dataset['sentence2']:
    flatten = []
    word_sent = row.split()
    tokenizer = Tokenizer( filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')
    tokenizer.fit_on_texts(word_sent)
    word_sent_dict = tokenizer.word_index
    hypothesis_sentence_length = np.append(hypothesis_sentence_length,len(word_sent_dict))
    #print(neg)
    #remove words that do not have word vector in glove
    t=0
    for word,i in word_sent_dict.items():
      
      embedding_vector = word2vec.get(word)

      if embedding_vector is not None:
        # words not found in embedding index will be all zeros.
        flatten= np.asarray(flatten)
        #print(premise_1.shape)
        flatten = np.append(flatten,embedding_vector)
        t= t+1
        
        
    #print(flatten.shape) 
    zero_pad = 15 - t
    arr = [0]*50*zero_pad
    flatten = np.append(flatten, arr)
    #print(flatten.shape) 
    hypothesis_embedding_matrix = np.vstack((hypothesis_embedding_matrix,flatten))
  hypothesis_embedding_matrix = np.delete(hypothesis_embedding_matrix, 0, 0)
  #convert labels to integers
  target = np.array(dataset['gold_label'])
  #target.reshape(1000,1)
  for index,i in enumerate(target):
    if i == 'neutral':
      target[index] = 0
    elif i == 'entailment':
      target[index] = 1
    elif i == 'contradiction':
      target[index] = 2

  length = len(target)
  one_hot_target = np.empty((3,), int)
  for index, i in enumerate(target):
    if i == 0:
      c = [1,0,0]
    elif i == 1:
      c = [0,1,0]
    elif i == 2:
      c = [0,0,1]
    one_hot_target = np.vstack([one_hot_target,c])
      
  
        
  print('shape of premise embeddings',premise_embedding_matrix.shape)
  print('shape of hypothesis embeddings',hypothesis_embedding_matrix.shape)
  return premise_embedding_matrix, hypothesis_embedding_matrix, one_hot_target

 










In [0]:
print('dev')
premise_embedding_matrix_dev, hypothesis_embedding_matrix_dev, target_dev  = word_embeddings(dev) 
print('train')
premise_embedding_matrix_train, hypothesis_embedding_matrix_train, target_train = word_embeddings(train) 
print('test')
premise_embedding_matrix_test, hypothesis_embedding_matrix_test, target_test  = word_embeddings(test) 



dev
shape of premise embeddings (1000, 750)
shape of hypothesis embeddings (1000, 750)
train
shape of premise embeddings (10000, 750)
shape of hypothesis embeddings (10000, 750)
test
shape of premise embeddings (990, 750)
shape of hypothesis embeddings (990, 750)


In [0]:
def remove_first(arr):
  arr = np.delete(arr, 0, 0)
  return arr
  
  
target_dev = remove_first(target_dev)
target_train = remove_first(target_train)
target_test = remove_first(target_test)
print(target_dev.shape)
print(target_train.shape)
print(target_test.shape)


(1000, 3)
(10000, 3)
[[1 0 0]
 [0 1 0]
 [0 1 0]
 ...
 [1 0 0]
 [0 1 0]
 [0 1 0]]


In [0]:
#normalizing the datasets
scaler = StandardScaler()
premise_embedding_matrix_dev = scaler.fit_transform(premise_embedding_matrix_dev)
hypothesis_embedding_matrix_dev = scaler.fit_transform(hypothesis_embedding_matrix_dev)
premise_embedding_matrix_train = scaler.fit_transform(premise_embedding_matrix_train)
hypothesis_embedding_matrix_train = scaler.fit_transform(hypothesis_embedding_matrix_train)
premise_embedding_matrix_test = scaler.fit_transform(premise_embedding_matrix_test)
hypothesis_embedding_matrix_test = scaler.fit_transform(hypothesis_embedding_matrix_test)



In [0]:
#model 1
pre_inputs = Input(shape=(750,))
x = Dense(750, activation='relu')(pre_inputs)
x = Dense(500, activation='relu')(x)
x = Dense(100, activation='sigmoid')(x)

premise_output = x

hyp_inputs = Input(shape=(750,))
y = Dense(750, activation='relu')(hyp_inputs)
y = Dense(500, activation='relu')(y)
y = Dense(100, activation='sigmoid')(y)

hypothesis_output = y

g = keras.layers.concatenate([premise_output, hypothesis_output])

g = Dense(200, activation='relu')(g)
g = Dense(100, activation='relu')(g)
main_output = Dense(3, activation='relu', name='main_output')(g)

model = Model(inputs=[pre_inputs,hyp_inputs], outputs=[main_output])
opt = optimizers.SGD(lr=0.20, decay=1e-2, momentum=0.9, nesterov=True)
model.compile(optimizer=opt, loss='mean_squared_error', loss_weights=[0.6], metrics=['accuracy'])




In [0]:
model.fit([premise_embedding_matrix_dev, hypothesis_embedding_matrix_dev], target_dev,epochs=10, batch_size=30)
model.fit([premise_embedding_matrix_train, hypothesis_embedding_matrix_train], target_train,epochs=10, batch_size=30)
model.evaluate([premise_embedding_matrix_test, hypothesis_embedding_matrix_test], target_test, verbose=0)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2366 - acc: 0.2930
Epoch 2/10
1000/1000 [==============================] - 1s 731us/step - loss: 0.2000 - acc: 0.2800
Epoch 3/10
1000/1000 [==============================] - 1s 741us/step - loss: 0.2000 - acc: 0.2800
Epoch 4/10
1000/1000 [==============================] - 1s 743us/step - loss: 0.2000 - acc: 0.2800
Epoch 5/10
1000/1000 [==============================] - 1s 739us/step - loss: 0.2000 - acc: 0.2800
Epoch 6/10
1000/1000 [==============================] - 1s 732us/step - loss: 0.2000 - acc: 0.2800
Epoch 7/10
1000/1000 [==============================] - 1s 733us/step - loss: 0.2000 - acc: 0.2800
Epoch 8/10
1000/1000 [==============================] - 1s 732us/step - loss: 0.2000 - acc: 0.2800
Epoch 9/10
1000/1000 [==============================] - 1s 748us/step - loss: 0.2000 - acc: 0.2800
Epoch 10/10
1000/1000 [==============================] - 1s 727

[0.20000001788139343, 0.27676767712891703]

In [0]:
#model 2
pre_inputs = Input(shape=(750,))
x = Dense(750, activation='sigmoid')(pre_inputs)
x = Dense(500, activation='sigmoid')(x)
x = Dense(100, activation='sigmoid')(x)

premise_output = x

hyp_inputs = Input(shape=(750,))
y = Dense(750, activation='sigmoid')(hyp_inputs)
y = Dense(500, activation='sigmoid')(y)
y = Dense(100, activation='sigmoid')(y)

hypothesis_output = y

g = keras.layers.concatenate([premise_output, hypothesis_output])

g = Dense(200, activation='sigmoid')(g)
g = Dense(50, activation='sigmoid')(g)
main_output = Dense(3, activation='relu', name='main_output')(g)

model2 = Model(inputs=[pre_inputs,hyp_inputs], outputs=[main_output])
opt = optimizers.SGD(lr=0.20, decay=1e-2, momentum=0.9, nesterov=True)
model2.compile(optimizer=opt, loss='mean_squared_error', loss_weights=[0.6], metrics=['accuracy'])


In [0]:
model2.fit([premise_embedding_matrix_dev, hypothesis_embedding_matrix_dev], target_dev,epochs=10, batch_size=30)
model2.fit([premise_embedding_matrix_train, hypothesis_embedding_matrix_train], target_train,epochs=10, batch_size=30)
model2.evaluate([premise_embedding_matrix_test, hypothesis_embedding_matrix_test], target_test, verbose=0)

Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1913 - acc: 0.3800
Epoch 1/10
10000/10000 [=====

[0.1944971103668213, 0.371]

In [0]:
#model 3
pre_inputs = Input(shape=(750,))
x = Dense(750, activation='sigmoid')(pre_inputs)
x = Dense(500, activation='sigmoid')(x)
x = Dense(300, activation='sigmoid')(x)

premise_output = x

hyp_inputs = Input(shape=(750,))
y = Dense(750, activation='sigmoid')(hyp_inputs)
y = Dense(500, activation='sigmoid')(y)
y = Dense(300, activation='sigmoid')(y)

hypothesis_output = y

g = keras.layers.concatenate([premise_output, hypothesis_output])

g = Dense(200, activation='sigmoid')(g)
g = Dense(100, activation='sigmoid')(g)
main_output = Dense(3, activation='relu', name='main_output')(g)

model3 = Model(inputs=[pre_inputs,hyp_inputs], outputs=[main_output])
opt = optimizers.SGD(lr=0.20, decay=1e-2, momentum=0.9, nesterov=True)
model3.compile(optimizer=opt, loss='mean_squared_error', loss_weights=[0.6], metrics=['accuracy'])


In [0]:
model3.fit([premise_embedding_matrix_dev, hypothesis_embedding_matrix_dev], target_dev,epochs=10, batch_size=30)
model3.fit([premise_embedding_matrix_train, hypothesis_embedding_matrix_train], target_train,epochs=10, batch_size=30)
model3.evaluate([premise_embedding_matrix_test, hypothesis_embedding_matrix_test], target_test, verbose=0)

Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 1.5220 - acc: 0.3320
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 1/10
10000/10000 [=====

[0.9042000379562378, 0.345]

In [0]:
#model 4
pre_inputs = Input(shape=(750,))
x = Dense(750, activation='sigmoid')(pre_inputs)
x = Dense(500, activation='sigmoid')(x)
x = Dense(300, activation='sigmoid')(x)

premise_output = x

hyp_inputs = Input(shape=(750,))
y = Dense(750, activation='sigmoid')(hyp_inputs)
y = Dense(500, activation='sigmoid')(y)
y = Dense(300, activation='sigmoid')(y)

hypothesis_output = y

g = keras.layers.concatenate([premise_output, hypothesis_output])

g = Dense(200, activation='sigmoid')(g)
g = Dense(50, activation='sigmoid')(g)
main_output = Dense(3, activation='relu', name='main_output')(g)

model4 = Model(inputs=[pre_inputs,hyp_inputs], outputs=[main_output])
opt = optimizers.SGD(lr=0.20, decay=1e-2, momentum=0.9, nesterov=True)
model4.compile(optimizer=opt, loss='mean_squared_error', loss_weights=[0.6], metrics=['accuracy'])



In [0]:
model4.fit([premise_embedding_matrix_dev, hypothesis_embedding_matrix_dev], target_dev,epochs=10, batch_size=30)
model4.fit([premise_embedding_matrix_train, hypothesis_embedding_matrix_train], target_train,epochs=10, batch_size=30)
model4.evaluate([premise_embedding_matrix_test, hypothesis_embedding_matrix_test], target_test, verbose=0)

Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9000 - acc: 0.3400
Epoch 1/10
10000/10000 [=====

[0.9042000379562378, 0.345]